# Le problème est : 

```
Traceback (most recent call last):
  File "/scratch/cnt0024/hmg2840/albert7a/DEV/git/diags-CMEMS-on-occigen/Profiles-EN4/2020-01-29-AA-process-profiles-regional-simus.py", line 408, in <module>
    sys.exit(main() or 0)
  File "/scratch/cnt0024/hmg2840/albert7a/DEV/git/diags-CMEMS-on-occigen/Profiles-EN4/2020-01-29-AA-process-profiles-regional-simus.py", line 397, in main
    process(jsonfile,infos,prof,dsN,latN,lonN,timN,config,case,namezone)
  File "/scratch/cnt0024/hmg2840/albert7a/DEV/git/diags-CMEMS-on-occigen/Profiles-EN4/2020-01-29-AA-process-profiles-regional-simus.py", line 51, in process
    checkb,profil_temp_model_mean,profil_temp_model_percent10,profil_temp_model_percent90,profil_salt_model_mean,profil_salt_model_percent10,profil_salt_model_percent90,observation_dep,obsred_temp,obsred_salt,dep_level,model_dep,obsred_dep,observation_lat,observation_lon,observation_time = model_mean_percent_profile(file_prof,prof_prof,dsN,latN,lonN,timN)
  File "/scratch/cnt0024/hmg2840/albert7a/DEV/git/diags-CMEMS-on-occigen/Profiles-EN4/2020-01-29-AA-process-profiles-regional-simus.py", line 235, in model_mean_percent_profile
    lat_near_observation,lon_near_observation,temp_model_mean,temp_percentile_10,temp_percentile_90,salt_model_mean,salt_percentile_10,salt_percentile_90=profile_mean_percent(int(model_level[int(z)]))
  File "/scratch/cnt0024/hmg2840/albert7a/DEV/git/diags-CMEMS-on-occigen/Profiles-EN4/2020-01-29-AA-process-profiles-regional-simus.py", line 218, in profile_mean_percent
    temp_percentile_10= np.percentile(model_temp_dask_concat,10)
  File "<__array_function__ internals>", line 6, in percentile
  File "/scratch/cnt0024/hmg2840/albert7a/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py", line 3713, in percentile
    a, q, axis, out, overwrite_input, interpolation, keepdims)
  File "/scratch/cnt0024/hmg2840/albert7a/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py", line 3833, in _quantile_unchecked
    interpolation=interpolation)
  File "/scratch/cnt0024/hmg2840/albert7a/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py", line 3410, in _ureduce
    r = func(a, **kwargs)
  File "/scratch/cnt0024/hmg2840/albert7a/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py", line 3948, in _quantile_ureduce_func
    x1 = take(ap, indices_below, axis=axis) * weights_below
  File "<__array_function__ internals>", line 6, in take
  File "/scratch/cnt0024/hmg2840/albert7a/anaconda3/lib/python3.7/site-packages/numpy/core/fromnumeric.py", line 194, in take
    return _wrapfunc(a, 'take', indices, axis=axis, out=out, mode=mode)
  File "/scratch/cnt0024/hmg2840/albert7a/anaconda3/lib/python3.7/site-packages/numpy/core/fromnumeric.py", line 61, in _wrapfunc
    return bound(*args, **kwds)
IndexError: cannot do a non-empty take from an empty axes.
```

In [1]:
import warnings
warnings.filterwarnings('ignore')


## imports

import numpy as np
import dask
import xarray as xr
from netCDF4 import Dataset
import matplotlib.pyplot as plt
import pandas as pd
import datetime
import glob as glob
import matplotlib.gridspec as gridspec
import time
from dask.diagnostics import ProgressBar
from datetime import date
import yaml
import io
import json
import os
import sys



In [2]:
def process(jsonfile,infos,prof,dsN,latN,lonN,timN,config,case,namezone):
        ''' Make all the steps needed to create the final netcdf files for one profile
        '''
        list_profiles = infos.keys()
        reference =  str(list(list_profiles)[prof])
        print('Processing profile ', reference)
        lat_prof = infos[list(list_profiles)[prof]]['latitude']
        lon_prof = infos[list(list_profiles)[prof]]['longitude']
        date_prof = infos[list(list_profiles)[prof]]['date']
        file_prof = infos[list(list_profiles)[prof]]['file']
        prof_prof = infos[list(list_profiles)[prof]]['profile no']

        check=check_prof_boundaries(dsN,latN,lonN,timN,lat_prof,lon_prof,date_prof)
        if check == 0.:
                checkb,profil_temp_model_mean,profil_temp_model_percent10,profil_temp_model_percent90,profil_salt_model_mean,profil_salt_model_percent10,profil_salt_model_percent90,observation_dep,obsred_temp,obsred_salt,dep_level,model_dep,obsred_dep,observation_lat,observation_lon,observation_time = model_mean_percent_profile(file_prof,prof_prof,dsN,latN,lonN,timN)
                if checkb == 0:
                    return
                else:
                    create_netcdf_profile(jsonfile,reference,prof_prof,observation_lat,observation_lon,observation_time,profil_temp_model_mean,profil_temp_model_percent10,profil_temp_model_percent90,profil_salt_model_mean,profil_salt_model_percent10,profil_salt_model_percent90,observation_dep,obsred_temp,obsred_salt,dep_level,model_dep,obsred_dep,config,case,namezone)



In [3]:
def open_model(config,case):
        print('Opening data')
        tfiles="/store/colombo/"+config+"/"+config+"-"+case+"-S/1d/2012/"+config+"-"+case+"_y20??m??d??.1d_gridT.nc"

        ##Open NATL60 files to get boundaries of domain

        dsN = xr.open_mfdataset(tfiles,decode_times=False, chunks={'deptht':1 ,'time_counter':10})

        latN = dsN.nav_lat
        lonN = dsN.nav_lon
        timN = dsN.time_counter
        return dsN,latN,lonN,timN



In [8]:
def check_prof_boundaries(dsN,latN,lonN,timN,lat_prof,lon_prof,date_prof):
        ''' Check if the selected profile falls within model boundaries
        '''

        lamin=np.nanmin(latN.values)
        lamax=np.nanmax(latN.values)
        lomin=np.nanmin(lonN.values)
        lomax=np.nanmax(lonN.values)

        if (lamin < lat_prof < lamax) & (lomin < lon_prof < lomax) :
                check=0.
                print("selected profile falls within model boundaries, the program is proceeding")
        else:
                check=1.
                print("selected profile does not fall within model boundaries, the program is stopping")
        distance_threshold = 0.25
        square_distance_to_observation = (lonN - lon_prof)**2 + (latN-lat_prof)**2
        is_close_to_observation = square_distance_to_observation < distance_threshold**2
        where_true=np.where(is_close_to_observation==True)
        if len(where_true[0]) < 1:
                check=1.
                print("there is no point in the model close enough to the profile, the program is stopping")
        else:
                check=0.
                print("there is a point in the model close enough to the profile, the program is proceeding")
        return check


In [5]:
def model_mean_percent_profile(fileEN4,ref_prof,dsN,latN,lonN,timN):
        #select NATL60 data at the closest depth and within 0.25 and 15days near the location and date of the ARGO profile
        diren4="/scratch/cnt0024/hmg2840/albert7a/EN4/"
        tfileEN4=diren4+fileEN4

        dsen4=xr.open_dataset(tfileEN4)

        laten4=dsen4['LATITUDE'][ref_prof]
        lonen4=dsen4['LONGITUDE'][ref_prof]
        dayen4=dsen4['JULD'][ref_prof]

        dateen4= pd.to_datetime(str(dayen4.values))
        ten4 = datetime.datetime(int(dateen4.strftime('%Y')),int(dateen4.strftime('%m')),int(dateen4.strftime('%d')),int(dateen4.strftime('%H')),int(dateen4.strftime('%M')))
        tsecen4=(ten4-datetime.datetime(1900,1,1,0,0)).total_seconds()

        observation_lon=lonen4
        observation_lat=laten4
        observation_time=tsecen4

        tempen4=dsen4['POTM_CORRECTED'][ref_prof]
        salten4=dsen4['PSAL_CORRECTED'][ref_prof]
        depen4=dsen4['DEPH_CORRECTED'][ref_prof]

        observation_dep=depen4
        observation_temp=tempen4
        observation_salt=salten4

        depN = dsN.deptht
        tempN=dsN.votemper
        saltN=dsN.vosaline

        #get the number of useful levels in EN4 profile
        dep_level=np.zeros(1)

        for k in np.arange(len(observation_dep)):
                if not np.isnan(observation_dep[k]):
                        dep_level[0]=k

        #get the corresponding model level
        model_level=np.zeros(int(dep_level[0]))
        model_dep=np.zeros(int(dep_level[0]))
        obsred_dep=np.zeros(int(dep_level[0]))
        obsred_temp=np.zeros(int(dep_level[0]))
        obsred_salt=np.zeros(int(dep_level[0]))
        for z in np.arange(int(dep_level[0])):
                obsred_dep[int(z)]=observation_dep[int(z)]
                obsred_temp[int(z)]=observation_temp[int(z)]
                obsred_salt[int(z)]=observation_salt[int(z)]
                diff_dep=0*depN.values
                for k in np.arange(len(depN.values)):
                        diff_dep[k]=depN.values[k]-obsred_dep[int(z)]
                lev=np.where(np.abs(diff_dep)==np.min(np.abs(diff_dep)))
                model_level[z]=lev[0]
                model_dep[z]=depN.values[lev[0]]

        #Coarse box in which EN4 profile is contained
        step=1
        indxBOX=np.where((lonN>observation_lon-1)&(lonN<observation_lon+1)&(latN>observation_lat-1)&(latN<observation_lat+1))
        if len(indxBOX[0]) < 1:
                print('profile is not really in the domain; exiting')
                return 0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

        model_lonBOX=lonN[np.min(indxBOX[0]):np.max(indxBOX[0]):step,np.min(indxBOX[1]):np.max(indxBOX[1]):step]
        model_latBOX=latN[np.min(indxBOX[0]):np.max(indxBOX[0]):step,np.min(indxBOX[1]):np.max(indxBOX[1]):step]
        model_lonBOX_array=model_lonBOX.values
        model_latBOX_array=model_latBOX.values
        t01012013 = datetime.datetime(2013,1,1,0,0)
        tsec01012013=(t01012013-datetime.datetime(1900,1,1,0,0)).total_seconds()
        if tsecen4 > tsec01012013:
                tsecen4=tsecen4-365*24*3600
        else:
                tsecen4=tsecen4
        indtBOX=np.where((timN.values < tsecen4 + 15*24*3600) & (timN.values > tsecen4 - 15*24*3600) )
        model_tBOX=timN[indtBOX[0][0]:indtBOX[0][-1]]
        t_dim=np.arange(len(model_tBOX))
        x_dim=np.arange(model_lonBOX_array.shape[1])
        y_dim=np.arange(model_lonBOX_array.shape[0])
        def profile_mean_percent(k):

                #decoupage grossier autour de la position du profile ARGO pour un niveau vertical

                model_tempBOX=tempN[indtBOX[0][0]:indtBOX[0][-1],k,np.min(indxBOX[0]):np.max(indxBOX[0]):step,np.min(indxBOX[1]):np.max(indxBOX[1]):step]
                model_saltBOX=saltN[indtBOX[0][0]:indtBOX[0][-1],k,np.min(indxBOX[0]):np.max(indxBOX[0]):step,np.min(indxBOX[1]):np.max(indxBOX[1]):step]

                model_tempBOX_array=model_tempBOX.values
                model_saltBOX_array=model_saltBOX.values
                model_tBOX_array=model_tBOX.values

                # construction d'un nouveau xarray
                d = {}
                d['time_counter'] = ('time_counter',t_dim)
                d['y'] = ('y',y_dim)
                d['x'] = ('x',x_dim)
                d['nav_lat'] = (['y','x'],model_latBOX_array)
                d['nav_lon'] = (['y','x'],model_lonBOX_array)

                d['votemper'] = (['time_counter','y','x'], model_tempBOX_array)
                d['vosaline'] = (['time_counter','y','x'], model_saltBOX_array)
                d['time_counter'] = (['time_counter'], model_tBOX_array)
                dset = xr.Dataset(d)

                latB = dset.nav_lat
                lonB = dset.nav_lon
                model_temperatureB = dset.votemper
                model_salinityB = dset.vosaline
                model_timeB = dset.time_counter

                # selection plus fine des profils

                lon_stacked = lonB.stack(profile=('x', 'y'))
                lat_stacked = latB.stack(profile=('x', 'y'))

                distance_threshold = 0.25
                square_distance_to_observation = (lon_stacked - observation_lon)**2 + (lat_stacked-observation_lat)**2
                is_close_to_observation = square_distance_to_observation < distance_threshold**2

                model_temperature_stacked = model_temperatureB.stack(profile=('x', 'y'))
                model_salinity_stacked = model_salinityB.stack(profile=('x', 'y'))

                model_temperature_near_observation = model_temperature_stacked.where(is_close_to_observation,drop=True)
                model_salinity_near_observation = model_salinity_stacked.where(is_close_to_observation, drop=True)
                lat_near_observation = lat_stacked.where(is_close_to_observation, drop=True)
                lon_near_observation = lon_stacked.where(is_close_to_observation, drop=True)

                model_temp_dask=dask.array.from_array(model_temperature_near_observation,chunks=(100,100))
                model_temp_dask_concat=dask.array.concatenate(model_temp_dask)
                model_salt_dask=dask.array.from_array(model_salinity_near_observation,chunks=(100,100))
                model_salt_dask_concat=dask.array.concatenate(model_salt_dask)
                temp_model_mean = model_temp_dask_concat.mean().compute()
                temp_percentile_10= np.percentile(model_temp_dask_concat,10)
                temp_percentile_90= np.percentile(model_temp_dask_concat,90)
                salt_model_mean = model_salt_dask_concat.mean().compute()
                salt_percentile_10= np.percentile(model_salt_dask_concat,10)
                salt_percentile_90= np.percentile(model_salt_dask_concat,90)

                return lat_near_observation,lon_near_observation,temp_model_mean,temp_percentile_10,temp_percentile_90,salt_model_mean,salt_percentile_10,salt_percentile_90

        profil_temp_model_mean=np.zeros(int(dep_level[0]))
        profil_temp_model_percent10=np.zeros(int(dep_level[0]))
        profil_temp_model_percent90=np.zeros(int(dep_level[0]))
        profil_salt_model_mean=np.zeros(int(dep_level[0]))
        profil_salt_model_percent10=np.zeros(int(dep_level[0]))
        profil_salt_model_percent90=np.zeros(int(dep_level[0]))


        for z in np.arange(dep_level[0]):
                lat_near_observation,lon_near_observation,temp_model_mean,temp_percentile_10,temp_percentile_90,salt_model_mean,salt_percentile_10,salt_percentile_90=profile_mean_percent(int(model_level[int(z)]))
                profil_temp_model_mean[int(z)]=temp_model_mean
                profil_temp_model_percent10[int(z)]=temp_percentile_10
                profil_temp_model_percent90[int(z)]=temp_percentile_90
                profil_salt_model_mean[int(z)]=salt_model_mean
                profil_salt_model_percent10[int(z)]=salt_percentile_10
                profil_salt_model_percent90[int(z)]=salt_percentile_90

        return 1,profil_temp_model_mean,profil_temp_model_percent10,profil_temp_model_percent90,profil_salt_model_mean,profil_salt_model_percent10,profil_salt_model_percent90,observation_dep,obsred_temp,obsred_salt,dep_level,model_dep,obsred_dep,observation_lat,observation_lon,observation_time



In [6]:
def create_netcdf_profile(jsonfile,reference,ref_prof,observation_lat,observation_lon,observation_time,profil_temp_model_mean,profil_temp_model_percent10,profil_temp_model_percent90,profil_salt_model_mean,profil_salt_model_percent10,profil_salt_model_percent90,observation_dep,obsred_temp,obsred_salt,dep_level,model_dep,obsred_dep,config,case,namezone):

        reference_profile=reference[-16:-1]
        dirname="/scratch/cnt0024/hmg2840/albert7a/EN4/profiles_files/"+namezone+"/"+config+"-"+case
        if not os.path.exists(dirname):
                os.mkdir(dirname)

        outname="/scratch/cnt0024/hmg2840/albert7a/EN4/profiles_files/"+namezone+"/"+config+"-"+case+"/profiles_EN4-"+reference_profile+"_"+config+"-"+case+"_TS.nc"
        print('output file is '+outname)
        dsout=Dataset(outname,'w')

        today=date.today()
        dsout.description = "This file contains one profile of temperature and salinity from EN4 dataset and the mean and 10 and 90 percentile of NATL60-CJM165 data within a 0.25deg circle around the location of the profile and 15 days before and after it has been sampled. This file has been created "+str(today.day)+"/"+str(today.month)+"/"+str(today.year)
        
        depth=dsout.createDimension('depth',dep_level[0])
        x=dsout.createDimension('x',1)
        y=dsout.createDimension('y',1)
        lat = dsout.createVariable('latitude_profileEN4', 'f8', ('y','x'))
        lat.standart_name="latitude_profileEN4"
        lat.long_name = "Latitude of selected EN4 profile"
        lat.units = "degrees_north"

        lon = dsout.createVariable('longitude_profileEN4', 'f8', ('y','x'))
        lon.standart_name="longitude_profileEN4"
        lon.long_name = "Longitude of selected EN4 profile"
        lon.units = "degrees_east"

        time = dsout.createVariable('time_profileEN4', 'f8', ('y','x'))
        time.standart_name="time_profileEN4"
        time.timeg_name = "Time in seconds from 1-1-1958 of selected EN4 profile"
        time.units = "seconds"

        depth_en4 = dsout.createVariable('depth_en4', 'f8', ('depth'),fill_value=0.)
        depth_en4.units = "m"
        depth_en4.valid_min = 0.
        depth_en4.valid_max = 8000.
        depth_en4.long_name = "Depth"

        depth_model = dsout.createVariable('depth_model', 'f8', ('depth'),fill_value=0.)
        depth_model.units = "m"
        depth_model.valid_min = 0.
        depth_model.valid_max = 8000.
        depth_model.long_name = "Depth"

        temp_en4 = dsout.createVariable('temp_profileEN4', 'f8', ('depth'),fill_value=0.)
        temp_en4.units = "degC"
        temp_en4.valid_min = -10.
        temp_en4.valid_max = 40.
        temp_en4.long_name = "Temperature profile of the selected EN4 profile"
        salt_en4 = dsout.createVariable('salt_profileEN4', 'f8', ('depth'),fill_value=0.)
        salt_en4.units = "PSU"
        salt_en4.valid_min = 20.
        salt_en4.valid_max = 40.
        salt_en4.long_name = "Salinity profile of the selected EN4 profile"

        mean_temp_model = dsout.createVariable('mean_temp_model', 'f8', ('depth'),fill_value=0.)
        mean_temp_model.units = "degC"
        mean_temp_model.valid_min = -10.
        mean_temp_model.valid_max = 40.
        mean_temp_model.long_name = "Mean Temperature profile of the model"

        mean_salt_model = dsout.createVariable('mean_salt_model', 'f8', ('depth'),fill_value=0.)
        mean_salt_model.units = "PSU"
        mean_salt_model.valid_min = 20.
        mean_salt_model.valid_max = 40.
        mean_salt_model.long_name = "Mean Salinity profile of the model"

        percent10_temp_model = dsout.createVariable('percent10_temp_model', 'f8', ('depth'),fill_value=0.)
        percent10_temp_model.units = "degC"
        percent10_temp_model.valid_min = -10.
        percent10_temp_model.valid_max = 40.
        percent10_temp_model.long_name = "Percent 10 Temperature profile of the model"

        percent10_salt_model = dsout.createVariable('percent10_salt_model', 'f8', ('depth'),fill_value=0.)
        percent10_salt_model.units = "PSU"
        percent10_salt_model.valid_min = 20.
        percent10_salt_model.valid_max = 40.
        percent10_salt_model.long_name = "Percent 10 Salinity profile of the model"

        percent90_temp_model = dsout.createVariable('percent90_temp_model', 'f8', ('depth'),fill_value=0.)
        percent90_temp_model.units = "degC"
        percent90_temp_model.valid_min = -90.
        percent90_temp_model.valid_max = 40.
        percent90_temp_model.long_name = "Percent 90 Temperature profile of the model"

        percent90_salt_model = dsout.createVariable('percent90_salt_model', 'f8', ('depth'),fill_value=0.)
        percent90_salt_model.units = "PSU"
        percent90_salt_model.valid_min = 20.
        percent90_salt_model.valid_max = 40.
        percent90_salt_model.long_name = "Percent 90 Salinity profile of the model"


        lat[:]=observation_lat
        lon[:]=observation_lon
        time[:]=observation_time
        depth_en4[:]=obsred_dep
        depth_model[:]=model_dep
        temp_en4[:]=obsred_temp
        salt_en4[:]=obsred_salt
        mean_temp_model[:]=profil_temp_model_mean
        mean_salt_model[:]=profil_salt_model_mean
        percent10_temp_model[:]=profil_temp_model_percent10
        percent10_salt_model[:]=profil_salt_model_percent10
        percent90_temp_model[:]=profil_temp_model_percent90
        percent90_salt_model[:]=profil_salt_model_percent90
        dsout.close()  # close the new file


In [17]:
        jsonfile = '/scratch/cnt0024/hmg2840/albert7a/EN4/profiles_files/NATL60-CJM165_EU_y2012-2013/NATL60-CJM165_EU_y2012-2013_73.json'
        dirn = '/scratch/cnt0024/hmg2840/albert7a/EN4/profiles_files/NATL60-CJM165_EU_y2012-2013/EU36-MPC001/'
        config = 'EU36'
        case = 'MPC001'
        namezone = 'NATL60-CJM165_EU_y2012-2013'

        sourcefile=open(jsonfile,'rU')
        infos=json.load(sourcefile)
        nb_profilesEN4=len(infos)

        print(time.strftime('%d/%m/%y %H:%M',time.localtime()))
        dsN,latN,lonN,timN = open_model(config,case)
        print(time.strftime('%d/%m/%y %H:%M',time.localtime()))

        prof=0
        list_profiles = infos.keys()
        reference = str(list(list_profiles)[prof])
        reference_profile=reference[-16:-1]
        print("dealing with profile "+reference_profile)
        outname=dirn+"/profiles_EN4-"+reference_profile+"_NATL60-CJM165_TS.nc"

        if not os.path.exists(outname):
            print(outname+" does not exist; lets process")
            process(jsonfile,infos,prof,dsN,latN,lonN,timN,config,case,namezone)
        else:
            print(outname+" already exists")

        print(time.strftime('%d/%m/%y %H:%M',time.localtime()))


04/02/20 15:36
Opening data
04/02/20 15:36
dealing with profile A20130209-40589
/scratch/cnt0024/hmg2840/albert7a/EN4/profiles_files/NATL60-CJM165_EU_y2012-2013/EU36-MPC001//profiles_EN4-A20130209-40589_NATL60-CJM165_TS.nc does not exist; lets process
Processing profile  b' A20130209-40589'
selected profile falls within model boundaries, the program is proceeding
profile is not really in the domain; exiting
04/02/20 15:37


In [7]:
        jsonfile = '/scratch/cnt0024/hmg2840/albert7a/EN4/profiles_files/NATL60-CJM165_EU_y2012-2013/NATL60-CJM165_EU_y2012-2013_73.json'
        dirn = '/scratch/cnt0024/hmg2840/albert7a/EN4/profiles_files/NATL60-CJM165_EU_y2012-2013/EU36-MPC001/'
        config = 'EU36'
        case = 'MPC001'
        namezone = 'NATL60-CJM165_EU_y2012-2013'

        sourcefile=open(jsonfile,'rU')
        infos=json.load(sourcefile)
        nb_profilesEN4=len(infos)

        print(time.strftime('%d/%m/%y %H:%M',time.localtime()))
        dsN,latN,lonN,timN = open_model(config,case)
        print(time.strftime('%d/%m/%y %H:%M',time.localtime()))

        prof=14
        list_profiles = infos.keys()
        reference = str(list(list_profiles)[prof])
        reference_profile=reference[-16:-1]
        print("dealing with profile "+reference_profile)
        outname=dirn+"/profiles_EN4-"+reference_profile+"_NATL60-CJM165_TS.nc"

        if not os.path.exists(outname):
            print(outname+" does not exist; lets process")
            process(jsonfile,infos,prof,dsN,latN,lonN,timN,config,case,namezone)
        else:
            print(outname+" already exists")

        print(time.strftime('%d/%m/%y %H:%M',time.localtime()))


04/02/20 15:29
Opening data
04/02/20 15:29
dealing with profile A20130209-40646
/scratch/cnt0024/hmg2840/albert7a/EN4/profiles_files/NATL60-CJM165_EU_y2012-2013/EU36-MPC001//profiles_EN4-A20130209-40646_NATL60-CJM165_TS.nc does not exist; lets process
Processing profile  b' A20130209-40646'
selected profile falls within model boundaries, the program is proceeding


IndexError: cannot do a non-empty take from an empty axes.

In [18]:
        jsonfile = '/scratch/cnt0024/hmg2840/albert7a/EN4/profiles_files/NATL60-CJM165_EU_y2012-2013/NATL60-CJM165_EU_y2012-2013_73.json'
        dirn = '/scratch/cnt0024/hmg2840/albert7a/EN4/profiles_files/NATL60-CJM165_EU_y2012-2013/EU36-MPC001/'
        config = 'EU36'
        case = 'MPC001'
        namezone = 'NATL60-CJM165_EU_y2012-2013'

        sourcefile=open(jsonfile,'rU')
        infos=json.load(sourcefile)
        nb_profilesEN4=len(infos)

        print(time.strftime('%d/%m/%y %H:%M',time.localtime()))
        dsN,latN,lonN,timN = open_model(config,case)
        print(time.strftime('%d/%m/%y %H:%M',time.localtime()))

        prof=14

04/02/20 15:38
Opening data
04/02/20 15:38


In [19]:
                list_profiles = infos.keys()
                reference = str(list(list_profiles)[prof])
                reference_profile=reference[-16:-1]
                print("dealing with profile "+reference_profile)
                outname=dirn+"/profiles_EN4-"+reference_profile+"_NATL60-CJM165_TS.nc"


dealing with profile A20130209-40646


In [20]:
        list_profiles = infos.keys()
        reference =  str(list(list_profiles)[prof])
        print('Processing profile ', reference)
        lat_prof = infos[list(list_profiles)[prof]]['latitude']
        lon_prof = infos[list(list_profiles)[prof]]['longitude']
        date_prof = infos[list(list_profiles)[prof]]['date']
        file_prof = infos[list(list_profiles)[prof]]['file']
        prof_prof = infos[list(list_profiles)[prof]]['profile no']



Processing profile  b' A20130209-40646'


In [21]:
check=check_prof_boundaries(dsN,latN,lonN,timN,lat_prof,lon_prof,date_prof)

selected profile falls within model boundaries, the program is proceeding


In [22]:
fileEN4=file_prof
ref_prof=prof_prof


In [23]:
        diren4="/scratch/cnt0024/hmg2840/albert7a/EN4/"
        tfileEN4=diren4+fileEN4

        dsen4=xr.open_dataset(tfileEN4)

        laten4=dsen4['LATITUDE'][ref_prof]
        lonen4=dsen4['LONGITUDE'][ref_prof]
        dayen4=dsen4['JULD'][ref_prof]

        dateen4= pd.to_datetime(str(dayen4.values))
        ten4 = datetime.datetime(int(dateen4.strftime('%Y')),int(dateen4.strftime('%m')),int(dateen4.strftime('%d')),int(dateen4.strftime('%H')),int(dateen4.strftime('%M')))
        tsecen4=(ten4-datetime.datetime(1900,1,1,0,0)).total_seconds()

        observation_lon=lonen4
        observation_lat=laten4
        observation_time=tsecen4

        tempen4=dsen4['POTM_CORRECTED'][ref_prof]
        salten4=dsen4['PSAL_CORRECTED'][ref_prof]
        depen4=dsen4['DEPH_CORRECTED'][ref_prof]

        observation_dep=depen4
        observation_temp=tempen4
        observation_salt=salten4

        depN = dsN.deptht
        tempN=dsN.votemper
        saltN=dsN.vosaline

        #get the number of useful levels in EN4 profile
        dep_level=np.zeros(1)

        for k in np.arange(len(observation_dep)):
                if not np.isnan(observation_dep[k]):
                        dep_level[0]=k

        #get the corresponding model level
        model_level=np.zeros(int(dep_level[0]))
        model_dep=np.zeros(int(dep_level[0]))
        obsred_dep=np.zeros(int(dep_level[0]))
        obsred_temp=np.zeros(int(dep_level[0]))
        obsred_salt=np.zeros(int(dep_level[0]))
        for z in np.arange(int(dep_level[0])):
                obsred_dep[int(z)]=observation_dep[int(z)]
                obsred_temp[int(z)]=observation_temp[int(z)]
                obsred_salt[int(z)]=observation_salt[int(z)]
                diff_dep=0*depN.values
                for k in np.arange(len(depN.values)):
                        diff_dep[k]=depN.values[k]-obsred_dep[int(z)]
                lev=np.where(np.abs(diff_dep)==np.min(np.abs(diff_dep)))
                model_level[z]=lev[0]
                model_dep[z]=depN.values[lev[0]]

        #Coarse box in which EN4 profile is contained
        step=1


In [50]:
        indxBOX=np.where((lonN>observation_lon-1)&(lonN<observation_lon+1)&(latN>observation_lat-1)&(latN<observation_lat+1))


In [51]:
print(indxBOX)

(array([399, 399, 399, 400, 400, 400, 401, 401, 401, 402, 402, 402, 403,
       403, 403, 404, 404, 404, 405, 405, 405, 406, 406, 406, 407, 407,
       407, 408, 408, 408, 409, 409, 409, 409, 410, 410, 410, 410, 411,
       411, 411, 411, 412, 412, 412, 412, 413, 413, 413, 413, 414, 414,
       414, 414, 415, 415, 415, 415, 416, 416, 416, 416, 417, 417, 417,
       417, 418, 418, 418, 418, 419, 419, 419, 419, 420, 420, 420, 420,
       421, 421, 421, 421, 422, 422, 422, 422, 423, 423, 423, 423, 424,
       424, 424, 424, 425, 425, 425, 425, 426, 426, 426, 426, 427, 427,
       427, 427, 428, 428, 428, 428, 429, 429, 429, 429, 430, 430, 430,
       430, 431, 431, 431, 431, 431, 432, 432, 432, 432, 432, 433, 433,
       433, 433, 433, 434, 434, 434, 434, 434, 435, 435, 435, 435, 435,
       436, 436, 436, 436, 436, 437, 437, 437, 437, 437, 438, 438, 438,
       438, 438, 439, 439, 439, 439, 439, 440, 440, 440, 440, 440, 441,
       441, 441, 441, 441, 442, 442, 442, 442, 442, 443, 443, 4

In [46]:
        indxBOX=np.where((lonN>observation_lon-0.03)&(lonN<observation_lon+0.03)&(latN>observation_lat-0.03)&(latN<observation_lat+0.03))


In [47]:
print(indxBOX)

(array([], dtype=int64), array([], dtype=int64))


In [48]:
        indxBOX=np.where((lonN>observation_lon-0.5)&(lonN<observation_lon+0.5)&(latN>observation_lat-0.5)&(latN<observation_lat+0.5))


In [49]:
print(indxBOX)

(array([], dtype=int64), array([], dtype=int64))


In [26]:
len(indxBOX[0])

556

In [27]:
        model_lonBOX=lonN[np.min(indxBOX[0]):np.max(indxBOX[0]):step,np.min(indxBOX[1]):np.max(indxBOX[1]):step]
        model_latBOX=latN[np.min(indxBOX[0]):np.max(indxBOX[0]):step,np.min(indxBOX[1]):np.max(indxBOX[1]):step]
        model_lonBOX_array=model_lonBOX.values
        model_latBOX_array=model_latBOX.values


In [28]:
        t01012013 = datetime.datetime(2013,1,1,0,0)
        tsec01012013=(t01012013-datetime.datetime(1900,1,1,0,0)).total_seconds()
        if tsecen4 > tsec01012013:
                tsecen4=tsecen4-365*24*3600
        else:
                tsecen4=tsecen4
        indtBOX=np.where((timN.values < tsecen4 + 15*24*3600) & (timN.values > tsecen4 - 15*24*3600) )
        model_tBOX=timN[indtBOX[0][0]:indtBOX[0][-1]]
        t_dim=np.arange(len(model_tBOX))
        x_dim=np.arange(model_lonBOX_array.shape[1])
        y_dim=np.arange(model_lonBOX_array.shape[0])


In [29]:
        def profile_mean_percent(k):

                #decoupage grossier autour de la position du profile ARGO pour un niveau vertical

                model_tempBOX=tempN[indtBOX[0][0]:indtBOX[0][-1],k,np.min(indxBOX[0]):np.max(indxBOX[0]):step,np.min(indxBOX[1]):np.max(indxBOX[1]):step]
                model_saltBOX=saltN[indtBOX[0][0]:indtBOX[0][-1],k,np.min(indxBOX[0]):np.max(indxBOX[0]):step,np.min(indxBOX[1]):np.max(indxBOX[1]):step]

                model_tempBOX_array=model_tempBOX.values
                model_saltBOX_array=model_saltBOX.values
                model_tBOX_array=model_tBOX.values

                # construction d'un nouveau xarray
                d = {}
                d['time_counter'] = ('time_counter',t_dim)
                d['y'] = ('y',y_dim)
                d['x'] = ('x',x_dim)
                d['nav_lat'] = (['y','x'],model_latBOX_array)
                d['nav_lon'] = (['y','x'],model_lonBOX_array)

                d['votemper'] = (['time_counter','y','x'], model_tempBOX_array)
                d['vosaline'] = (['time_counter','y','x'], model_saltBOX_array)
                d['time_counter'] = (['time_counter'], model_tBOX_array)
                dset = xr.Dataset(d)

                latB = dset.nav_lat
                lonB = dset.nav_lon
                model_temperatureB = dset.votemper
                model_salinityB = dset.vosaline
                model_timeB = dset.time_counter

                # selection plus fine des profils

                lon_stacked = lonB.stack(profile=('x', 'y'))
                lat_stacked = latB.stack(profile=('x', 'y'))

                distance_threshold = 0.25
                square_distance_to_observation = (lon_stacked - observation_lon)**2 + (lat_stacked-observation_lat)**2
                is_close_to_observation = square_distance_to_observation < distance_threshold**2

                model_temperature_stacked = model_temperatureB.stack(profile=('x', 'y'))
                model_salinity_stacked = model_salinityB.stack(profile=('x', 'y'))

                model_temperature_near_observation = model_temperature_stacked.where(is_close_to_observation,drop=True)
                model_salinity_near_observation = model_salinity_stacked.where(is_close_to_observation, drop=True)
                lat_near_observation = lat_stacked.where(is_close_to_observation, drop=True)
                lon_near_observation = lon_stacked.where(is_close_to_observation, drop=True)

                model_temp_dask=dask.array.from_array(model_temperature_near_observation,chunks=(100,100))
                model_temp_dask_concat=dask.array.concatenate(model_temp_dask)
                model_salt_dask=dask.array.from_array(model_salinity_near_observation,chunks=(100,100))
                model_salt_dask_concat=dask.array.concatenate(model_salt_dask)
                temp_model_mean = model_temp_dask_concat.mean().compute()
                temp_percentile_10= np.percentile(model_temp_dask_concat,10)
                temp_percentile_90= np.percentile(model_temp_dask_concat,90)
                salt_model_mean = model_salt_dask_concat.mean().compute()
                salt_percentile_10= np.percentile(model_salt_dask_concat,10)
                salt_percentile_90= np.percentile(model_salt_dask_concat,90)

                return lat_near_observation,lon_near_observation,temp_model_mean,temp_percentile_10,temp_percentile_90,salt_model_mean,salt_percentile_10,salt_percentile_90

        profil_temp_model_mean=np.zeros(int(dep_level[0]))
        profil_temp_model_percent10=np.zeros(int(dep_level[0]))
        profil_temp_model_percent90=np.zeros(int(dep_level[0]))
        profil_salt_model_mean=np.zeros(int(dep_level[0]))
        profil_salt_model_percent10=np.zeros(int(dep_level[0]))
        profil_salt_model_percent90=np.zeros(int(dep_level[0]))



In [30]:
print(dep_level[0])

16.0


In [31]:
                z=0
                lat_near_observation,lon_near_observation,temp_model_mean,temp_percentile_10,temp_percentile_90,salt_model_mean,salt_percentile_10,salt_percentile_90=profile_mean_percent(int(model_level[int(z)]))
                profil_temp_model_mean[int(z)]=temp_model_mean
                profil_temp_model_percent10[int(z)]=temp_percentile_10
                profil_temp_model_percent90[int(z)]=temp_percentile_90
                profil_salt_model_mean[int(z)]=salt_model_mean
                profil_salt_model_percent10[int(z)]=salt_percentile_10
                profil_salt_model_percent90[int(z)]=salt_percentile_90



IndexError: cannot do a non-empty take from an empty axes.

In [33]:
k=int(model_level[int(z)])

In [34]:
                model_tempBOX=tempN[indtBOX[0][0]:indtBOX[0][-1],k,np.min(indxBOX[0]):np.max(indxBOX[0]):step,np.min(indxBOX[1]):np.max(indxBOX[1]):step]
                model_saltBOX=saltN[indtBOX[0][0]:indtBOX[0][-1],k,np.min(indxBOX[0]):np.max(indxBOX[0]):step,np.min(indxBOX[1]):np.max(indxBOX[1]):step]

                model_tempBOX_array=model_tempBOX.values
                model_saltBOX_array=model_saltBOX.values
                model_tBOX_array=model_tBOX.values


In [36]:
                d = {}
                d['time_counter'] = ('time_counter',t_dim)
                d['y'] = ('y',y_dim)
                d['x'] = ('x',x_dim)
                d['nav_lat'] = (['y','x'],model_latBOX_array)
                d['nav_lon'] = (['y','x'],model_lonBOX_array)

                d['votemper'] = (['time_counter','y','x'], model_tempBOX_array)
                d['vosaline'] = (['time_counter','y','x'], model_saltBOX_array)
                d['time_counter'] = (['time_counter'], model_tBOX_array)
                dset = xr.Dataset(d)

                latB = dset.nav_lat
                lonB = dset.nav_lon
                model_temperatureB = dset.votemper
                model_salinityB = dset.vosaline
                model_timeB = dset.time_counter

                # selection plus fine des profils

                lon_stacked = lonB.stack(profile=('x', 'y'))
                lat_stacked = latB.stack(profile=('x', 'y'))

                distance_threshold = 0.25
                square_distance_to_observation = (lon_stacked - observation_lon)**2 + (lat_stacked-observation_lat)**2
                is_close_to_observation = square_distance_to_observation < distance_threshold**2

                model_temperature_stacked = model_temperatureB.stack(profile=('x', 'y'))
                model_salinity_stacked = model_salinityB.stack(profile=('x', 'y'))

                model_temperature_near_observation = model_temperature_stacked.where(is_close_to_observation,drop=True)
                model_salinity_near_observation = model_salinity_stacked.where(is_close_to_observation, drop=True)
                lat_near_observation = lat_stacked.where(is_close_to_observation, drop=True)
                lon_near_observation = lon_stacked.where(is_close_to_observation, drop=True)

                model_temp_dask=dask.array.from_array(model_temperature_near_observation,chunks=(100,100))
                model_temp_dask_concat=dask.array.concatenate(model_temp_dask)
                model_salt_dask=dask.array.from_array(model_salinity_near_observation,chunks=(100,100))
                model_salt_dask_concat=dask.array.concatenate(model_salt_dask)
                temp_model_mean = model_temp_dask_concat.mean().compute()


In [38]:
model_temp_dask_concat


dask.array<concatenate, shape=(0,), dtype=float32, chunksize=(0,)>

In [39]:
model_temperature_near_observation

<xarray.DataArray 'votemper' (time_counter: 29, profile: 0)>
array([], shape=(29, 0), dtype=float32)
Coordinates:
  * time_counter  (time_counter) float64 3.537e+09 3.537e+09 ... 3.539e+09
  * profile       (profile) MultiIndex
  - x             (profile) int64 
  - y             (profile) int64

In [40]:
model_temperature_stacked

<xarray.DataArray 'votemper' (time_counter: 29, profile: 707)>
array([[14.299382 , 14.282785 , 14.267995 , ..., 12.81181  , 12.765917 ,
        12.73861  ],
       [14.243149 , 14.224168 , 14.208095 , ..., 12.750693 , 12.703374 ,
        12.674894 ],
       [14.191671 , 14.172272 , 14.155804 , ..., 12.689578 , 12.640832 ,
        12.611179 ],
       ...,
       [13.305785 , 13.278197 , 13.2564745, ..., 11.827638 , 11.765399 ,
        11.722301 ],
       [13.239938 , 13.225489 , 13.2081995, ..., 11.867328 , 11.807121 ,
        11.7653   ],
       [13.4029875, 13.392927 , 13.37775  , ..., 11.907018 , 11.848845 ,
        11.8083   ]], dtype=float32)
Coordinates:
  * time_counter  (time_counter) float64 3.537e+09 3.537e+09 ... 3.539e+09
  * profile       (profile) MultiIndex
  - x             (profile) int64 0 0 0 0 0 0 0 0 0 0 0 ... 6 6 6 6 6 6 6 6 6 6
  - y             (profile) int64 0 1 2 3 4 5 6 7 8 ... 93 94 95 96 97 98 99 100

In [41]:
is_close_to_observation

<xarray.DataArray (profile: 707)>
array([False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, Fals

In [42]:
square_distance_to_observation

<xarray.DataArray (profile: 707)>
array([2.23773367, 2.1960118 , 2.15508987, 2.11497559, 2.075667  ,
       2.03715696, 1.99945168, 1.96254972, 1.92645074, 1.89114768,
       1.85664612, 1.82294464, 1.7900429 , 1.75793946, 1.72662858,
       1.69611507, 1.66639189, 1.63747011, 1.60933719, 1.58199894,
       1.55545342, 1.52970028, 1.50473339, 1.48055741, 1.45717147,
       1.43456989, 1.41276047, 1.39173449, 1.37149203, 1.3520379 ,
       1.33336584, 1.31547826, 1.29837324, 1.28205093, 1.26650706,
       1.25174445, 1.23776172, 1.22455606, 1.21212831, 1.20047862,
       1.18960612, 1.17950758, 1.17018427, 1.16163532, 1.15385934,
       1.14685575, 1.14062419, 1.1351633 , 1.13047242, 1.12655089,
       1.12339845, 1.12101365, 1.11939645, 1.11854547, 1.11846016,
       1.11913992, 1.12058301, 1.12279024, 1.12575947, 1.1294912 ,
       1.13398299, 1.13923586, 1.14524938, 1.15202217, 1.15955187,
       1.16783862, 1.17688155, 1.18668186, 1.19723643, 1.20854774,
       1.2206114 , 1.23342752, 1.24699523, 1.26131617, 1.27638728,
       1.2922077 , 1.30877561, 1.32609516, 1.34416126, 1.36297305,
       1.38253417, 1.40284005, 1.42389366, 1.44568566, 1.46822906,
       1.49150915, 1.51552904, 1.54029439, 1.56579814, 1.59204621,
       1.61903128, 1.64674684, 1.67521088, 1.70440275, 1.7343423 ,
       1.76501564, 1.79642146, 1.82855937, 1.86142853, 1.89503539,
       1.92937162, 2.17757054, 2.13587584, 2.09498905, 2.05490927,
       2.01562816, 1.97715259, 1.93948166, 1.90260803, 1.86653705,
       1.83127453, 1.79680107, 1.76313378, 1.73026612, 1.69819113,
       1.66690939, 1.63642947, 1.60674084, 1.5778475 , 1.54974908,
       1.52244472, 1.49592849, 1.47020869, 1.44527608, 1.42112973,
       1.39777701, 1.37520959, 1.35342963, 1.33243727, 1.31222833,
       1.29280502, 1.27416596, 1.25631179, 1.23923742, 1.22294851,
       1.20743796, 1.19270858, 1.1787569 , 1.16558444, 1.15318804,
       1.14157154, 1.13073015, 1.12066479, 1.11137457, 1.10285813,
       1.0951147 , 1.08814361, 1.08194438, 1.07651648, 1.07185851,
       1.0679697 , 1.06485015, 1.06249818, 1.06091355, 1.06009538,
       1.0600428 , 1.06075523, 1.06223182, 1.06447122, 1.06747377,
       1.0712373 , 1.07576255, 1.0810491 , 1.08709389, 1.0939005 ,
       1.10146266, 1.10978129, 1.11885892, 1.12869112, 1.1392807 ,
       1.15062439, 1.16272037, 1.17556874, 1.18917145, 1.20352464,
       1.21862794, 1.23448375, 1.25108386, 1.26843507, 1.28653696,
       1.3053809 , 1.32497411, 1.34531569, 1.36639708, 1.38823036,
       1.4108008 , 1.43411233, 1.45816946, 1.48296627, 1.50850756,
       1.53478712, 1.56180409, 1.5895576 , 1.61805263, 1.64728326,
       1.67725428, 1.70795858, 1.73939623, 1.77156592, 1.80447349,
       1.83811169, 1.87247967, 2.11897272, 2.07731267, 2.03645318,
       1.9964009 , 1.95715495, 1.91871393, 1.88107083, 1.84423169,
       1.80819512, 1.77296075, 1.73852768, 1.70488864, 1.67204944,
       1.64000919, 1.60876108, 1.57831045, 1.54865592, 1.51979163,
       1.49172729, 1.4644569 , 1.43797456, 1.4122842 , 1.38738545,
       1.36327286, 1.33994992, 1.31741574, 1.29566994, 1.27470769,
       1.25453188, 1.2351421 , 1.21653399, 1.19871283, 1.18167238,
       1.16541378, 1.14993713, 1.13523833, 1.12132003, 1.10818087,
       1.09581769, 1.08423268, 1.07342449, 1.06339176, 1.05413338,
       1.04564902, 1.03793876, 1.03100077, 1.02483406, 1.01943866,
       1.01481369, 1.01095829, 1.00787112, 1.00555207, 1.00400027,
       1.00321487, 1.0031951 , 1.00394029, 1.00544909, 1.00772216,
       1.0107574 , 1.01455442, 1.01911236, 1.02443152, 1.03051017,
       1.03734744, 1.04494409, 1.05329745, 1.06240714, 1.07227183,
       1.0828938 , 1.09426935, 1.1063976 , 1.11928083, 1.13291582,
       1.14730122, 1.16243978, 1.1783278 , 1.19496396, 1.21234695,
       1.230481  , 1.24936092, 1.2689863 , 1.28936001, 1.31047776,
       1.33233779, 1.35494524, 1.378294  , 1.40238277, 1.42721161,
       1.45278442, 1.47909589, 1.50615021, 1.53394132, 1.56246837,
       1.5917

In [43]:
distance_threshold**2

0.0625

In [44]:
lon_stacked

<xarray.DataArray 'nav_lon' (profile: 707)>
array([3.2354898, 3.2366772, 3.237868 , 3.239062 , 3.2402596, 3.2414606,
       3.2426648, 3.2438726, 3.2450838, 3.2462983, 3.2475162, 3.2487376,
       3.2499623, 3.2511907, 3.2524223, 3.2536573, 3.2548962, 3.256138 ,
       3.2573838, 3.2586327, 3.259885 , 3.2611408, 3.2624004, 3.2636633,
       3.2649295, 3.2661996, 3.267473 , 3.26875  , 3.2700303, 3.2713144,
       3.2726018, 3.2738926, 3.2751873, 3.2764852, 3.277787 , 3.279092 ,
       3.2804008, 3.281713 , 3.283029 , 3.2843485, 3.2856712, 3.2869978,
       3.288328 , 3.2896616, 3.2909992, 3.29234  , 3.2936845, 3.2950327,
       3.2963846, 3.2977402, 3.2990992, 3.300462 , 3.3018281, 3.303198 ,
       3.3045716, 3.305949 , 3.30733  , 3.3087144, 3.3101027, 3.3114948,
       3.3128903, 3.3142896, 3.3156927, 3.317099 , 3.3185096, 3.3199236,
       3.3213413, 3.3227627, 3.324188 , 3.3256168, 3.3270497, 3.3284862,
       3.3299263, 3.3313704, 3.332818 , 3.3342693, 3.3357246, 3.3371835,
       3.3386462, 3.3401127, 3.341583 , 3.343057 , 3.3445349, 3.3460164,
       3.347502 , 3.3489912, 3.3504844, 3.3519812, 3.353482 , 3.3549864,
       3.356495 , 3.3580072, 3.359523 , 3.3610432, 3.362567 , 3.3640945,
       3.365626 , 3.3671615, 3.3687007, 3.3702438, 3.371791 , 3.2634408,
       3.264629 , 3.2658203, 3.267015 , 3.2682133, 3.269415 , 3.2706199,
       3.2718282, 3.27304  , 3.2742553, 3.2754738, 3.276696 , 3.2779214,
       3.2791505, 3.2803826, 3.2816186, 3.282858 , 3.2841008, 3.285347 ,
       3.2865965, 3.2878497, 3.2891064, 3.2903664, 3.29163  , 3.2928972,
       3.2941678, 3.295442 , 3.2967198, 3.2980008, 3.2992854, 3.3005738,
       3.3018653, 3.3031607, 3.3044593, 3.3057618, 3.3070676, 3.308377 ,
       3.30969  , 3.3110068, 3.312327 , 3.3136506, 3.314978 , 3.3163087,
       3.3176434, 3.3189814, 3.3203232, 3.3216684, 3.3230174, 3.32437  ,
       3.3257263, 3.327086 , 3.3284495, 3.3298166, 3.3311872, 3.3325615,
       3.3339396, 3.3353212, 3.3367066, 3.338096 , 3.3394885, 3.340885 ,
       3.3422852, 3.3436887, 3.345096 , 3.3465073, 3.3479223, 3.3493407,
       3.350763 , 3.352189 , 3.3536186, 3.3550522, 3.3564894, 3.3579304,
       3.3593752, 3.3608236, 3.3622758, 3.3637319, 3.3651917, 3.3666553,
       3.3681226, 3.3695936, 3.3710687, 3.3725471, 3.3740296, 3.375516 ,
       3.377006 , 3.3785   , 3.3799977, 3.3814993, 3.3830047, 3.3845139,
       3.3860269, 3.387544 , 3.3890646, 3.3905892, 3.392118 , 3.3936503,
       3.3951864, 3.3967266, 3.3982706, 3.3998184, 3.2913914, 3.2925801,
       3.2937722, 3.2949677, 3.2961664, 3.2973688, 3.2985744, 3.2997835,
       3.300996 , 3.302212 , 3.3034313, 3.3046541, 3.3058803, 3.3071098,
       3.308343 , 3.3095794, 3.3108194, 3.312063 , 3.31331  , 3.3145602,
       3.315814 , 3.3170714, 3.3183322, 3.3195965, 3.3208644, 3.322136 ,
       3.3234107, 3.3246891, 3.3259711, 3.3272564, 3.3285453, 3.3298378,
       3.3311336, 3.3324332, 3.3337362, 3.335043 , 3.336353 , 3.3376667,
       3.3389843, 3.340305 , 3.3416295, 3.3429577, 3.3442893, 3.3456247,
       3.3469634, 3.348306 , 3.349652 , 3.3510017, 3.352355 , 3.3537118,
       3.3550725, 3.3564367, 3.3578045, 3.359176 , 3.360551 , 3.36193  ,
       3.3633125, 3.3646986, 3.3660886, 3.3674822, 3.3688793, 3.3702803,
       3.3716848, 3.373093 , 3.3745048, 3.3759205, 3.3773398, 3.378763 ,
       3.3801897, 3.3816202, 3.3830545, 3.3844926, 3.3859344, 3.38738  ,
       3.3888292, 3.3902822, 3.391739 , 3.3931997, 3.394664 , 3.3961322,
       3.397604 , 3.3990798, 3.4005592, 3.4020426, 3.4035296, 3.4050205,
       3.4065154, 3.4080138, 3.4095163, 3.4110224, 3.4125326, 3.4140465,
       3.4155643, 3.417086 , 3.4186113, 3.4201407, 3.421674 , 3.423211 ,
       3.424752 , 3.426297 , 3.4278457, 3.3193417, 3.3205311, 3.321724 ,
       3.32292  , 3.3241196, 3.3253226, 3.3265288, 3.3277385, 3.3289518,
       3.3301685, 3.3313885, 3.332612 , 3.3338387, 3.3350692, 3.3363028,
       3.33754  , 3.3387806, 3.340025 , 3.3412726, 3.3425238, 3.3437784,
       3.34503

In [45]:
observation_lon

<xarray.DataArray 'LONGITUDE' ()>
array(4.359)
Attributes:
    long_name:   longitude of the station, best estimated value
    units:       degree_east
    _fillvalue:  99999.0
    valid_min:   -180.0
    valid_max:   180.0

In [ ]:
        jsonfile = '/scratch/cnt0024/hmg2840/albert7a/EN4/profiles_files/NATL60-CJM165_EU_y2012-2013/NATL60-CJM165_EU_y2012-2013_281.json'
        dirn = '/scratch/cnt0024/hmg2840/albert7a/EN4/profiles_files/NATL60-CJM165_EU_y2012-2013/EU36-MPC001/'
        config = 'EU36'
        case = 'MPC001'
        namezone = 'NATL60-CJM165_EU_y2012-2013'

        sourcefile=open(jsonfile,'rU')
        infos=json.load(sourcefile)
        nb_profilesEN4=len(infos)

        print(time.strftime('%d/%m/%y %H:%M',time.localtime()))
        dsN,latN,lonN,timN = open_model(config,case)
        print(time.strftime('%d/%m/%y %H:%M',time.localtime()))

        for prof in np.arange(nb_profilesEN4):
            list_profiles = infos.keys()
            reference = str(list(list_profiles)[prof])
            reference_profile=reference[-16:-1]
            print("dealing with profile "+reference_profile)
            outname=dirn+"/profiles_EN4-"+reference_profile+"_NATL60-CJM165_TS.nc"
            list_profiles = infos.keys()
            reference =  str(list(list_profiles)[prof])
            print('Processing profile ', reference)
            lat_prof = infos[list(list_profiles)[prof]]['latitude']
            lon_prof = infos[list(list_profiles)[prof]]['longitude']
            date_prof = infos[list(list_profiles)[prof]]['date']
            file_prof = infos[list(list_profiles)[prof]]['file']
            prof_prof = infos[list(list_profiles)[prof]]['profile no']
            check=check_prof_boundaries(dsN,latN,lonN,timN,lat_prof,lon_prof,date_prof)


04/02/20 17:23
Opening data
04/02/20 17:23
dealing with profile A20130927-84875
Processing profile  b' A20130927-84875'
selected profile falls within model boundaries, the program is proceeding
there is a point in the model close enough to the profile, the program is proceeding
dealing with profile A20130927-84914
Processing profile  b' A20130927-84914'
selected profile falls within model boundaries, the program is proceeding
there is a point in the model close enough to the profile, the program is proceeding
dealing with profile A20130927-84971
Processing profile  b' A20130927-84971'
selected profile falls within model boundaries, the program is proceeding
there is a point in the model close enough to the profile, the program is proceeding
dealing with profile A20130927-84995
Processing profile  b' A20130927-84995'
selected profile falls within model boundaries, the program is proceeding
there is a point in the model close enough to the profile, the program is proceeding
dealing with 